In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from scipy.stats import linregress
import requests
import json
import gmaps
import os

# API Keyz
from api_keys import g_key
from api_keys import weather_api_key

# Data files
HUD_2019_path = "Data/HUD Exchange Data/2019-PIT-Counts-by-State.csv"
HUD_2018_path = "Data/HUD Exchange Data/2018-PIT-Counts-by-State.csv"
HUD_2017_path = "Data/HUD Exchange Data/2017-PIT-Counts-by-State.csv"
HUD_2016_path = "Data/HUD Exchange Data/2016-PIT-Counts-by-State.csv"
HUD_2015_path = "Data/HUD Exchange Data/2015-PIT-Counts-by-State.csv"

# Read csvs
hud_2019_data = pd.read_csv(HUD_2019_path)
hud_2018_data = pd.read_csv(HUD_2018_path)
hud_2017_data = pd.read_csv(HUD_2017_path)
hud_2016_data = pd.read_csv(HUD_2016_path)
hud_2015_data = pd.read_csv(HUD_2015_path)

In [13]:
new_hud_2019_data = hud_2019_data.dropna()
new_hud_2019_data.head()

,State,Number of CoCs,"Overall Homeless, 2019","Overall Homeless - Under 18, 2019","Overall Homeless - Age 18 to 24, 2019","Overall Homeless - Over 24, 2019","Overall Homeless - Female, 2019","Overall Homeless - Male, 2019","Overall Homeless - Transgender, 2019","Overall Homeless - Gender Non-Conforming, 2019",...,"Overall Homeless Parenting Youth Age 18-24, 2019","Sheltered ES Homeless Parenting Youth Age 18-24, 2019","Sheltered TH Homeless Parenting Youth Age 18-24, 2019","Sheltered Total Homeless Parenting Youth Age 18-24, 2019","Unsheltered Homeless Parenting Youth Age 18-24, 2019","Overall Homeless Children of Parenting Youth, 2019","Sheltered ES Homeless Children of Parenting Youth, 2019","Sheltered TH Homeless Children of Parenting Youth, 2019","Sheltered Total Homeless Children of Parenting Youth, 2019","Unsheltered Homeless Children of Parenting Youth, 2019"
0,AK,2.0,1907.0,335.0,204.0,1368.0,761.0,1137.0,6.0,3.0,...,20.0,6.0,14.0,20.0,0.0,27.0,13.0,14.0,27.0,0.0
1,AL,8.0,3261.0,629.0,199.0,2433.0,1383.0,1873.0,3.0,2.0,...,22.0,10.0,11.0,21.0,1.0,34.0,17.0,13.0,30.0,4.0
2,AR,5.0,2717.0,303.0,317.0,2097.0,1022.0,1688.0,4.0,3.0,...,11.0,7.0,4.0,11.0,0.0,15.0,10.0,5.0,15.0,0.0
3,AZ,3.0,10007.0,1602.0,708.0,7697.0,3683.0,6297.0,16.0,11.0,...,58.0,17.0,40.0,57.0,1.0,80.0,26.0,53.0,79.0,1.0
4,CA,44.0,151278.0,14305.0,12673.0,124300.0,50467.0,98404.0,1764.0,643.0,...,1019.0,457.0,291.0,748.0,271.0,1194.0,555.0,357.0,912.0,282.0


In [14]:
#pulling specific columns to work with
overall2019_df = new_hud_2019_data.loc[:, ["State","Overall Homeless, 2019"]]
overall2019_df

,State,"Overall Homeless, 2019"
0,AK,1907.0
1,AL,3261.0
2,AR,2717.0
3,AZ,10007.0
4,CA,151278.0
5,CO,9619.0
6,CT,3033.0
7,DC,6521.0
8,DE,921.0
9,FL,28328.0


In [15]:
new_overall2019_df = overall2019_df.drop([55], axis=0)
new_overall2019_df.head()

,State,"Overall Homeless, 2019"
0,AK,1907.0
1,AL,3261.0
2,AR,2717.0
3,AZ,10007.0
4,CA,151278.0


In [24]:
#add columns for lat and lng 
new_overall2019_df["Lat"] = ""
new_overall2019_df["Lng"] = ""
new_overall2019_df.head()

,State,"Overall Homeless, 2019",Lat,Lng,Temp
0,AK,1907.0,,,
1,AL,3261.0,,,
2,AR,2717.0,,,
3,AZ,10007.0,,,
4,CA,151278.0,,,


In [25]:
# create a params dict
params = {"key": g_key}

# Loop through and run a lat/long search for each city
for index, row in new_overall2019_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    
    state = row['State']

    # update address key value
    params['address'] = f"{state}"

    # make request
    states_lat_lng = requests.get(base_url, params=params)
    
    # convert to json
    states_lat_lng = states_lat_lng.json()

    new_overall2019_df.loc[index, "Lat"] = states_lat_lng["results"][0]["geometry"]["location"]["lat"]
    new_overall2019_df.loc[index, "Lng"] = states_lat_lng["results"][0]["geometry"]["location"]["lng"]

# Visualize to confirm lat lng appear
new_overall2019_df.head()

,State,"Overall Homeless, 2019",Lat,Lng,Temp
0,AK,1907.0,36.161,-95.736,
1,AL,3261.0,32.3182,-86.9023,
2,AR,2717.0,35.2011,-91.8318,
3,AZ,10007.0,34.0489,-111.094,
4,CA,151278.0,36.7783,-119.418,


In [31]:
#weather for states
temp = []

for i in range(len(new_overall2019_df)):
    lat = new_overall2019_df.loc[i]["Lat"]
    lng = new_overall2019_df.loc[i]["Lng"]

    params = {
        "location": f"{lat},{lng}",
    }
    
    url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key 

    response = requests.get(url, params=params).json()
    
    try:
        temp.append(response['results'][0]['temp'])
    except:
        temp.append("")
        
new_overall2019_df["Temp"] = temp
map2019 = new_overall2019_df.dropna()
map2019.head()

,State,"Overall Homeless, 2019",Lat,Lng,Temp
0,AK,1907.0,36.161,-95.736,
1,AL,3261.0,32.3182,-86.9023,
2,AR,2717.0,35.2011,-91.8318,
3,AZ,10007.0,34.0489,-111.094,
4,CA,151278.0,36.7783,-119.418,


In [ ]:
#coordinates are too large to get one temp! Silly me!